## Remap events for the Sternberg Data

This notebook assumes that you have already created a remapped template

In [1]:
import os
import datetime
from hed.tools import BidsTsvDictionary, KeyMap,  HedLogger
from hed.util import get_file_list, get_new_dataframe

# Variables to set for the specific dataset
bids_root_path = 'G:\Sternberg\SternbergWorking'
exclude_dirs = ['sourcedata', 'stimuli', 'code']
entities = ('sub', 'ses', 'run')
key_columns = ["value"]
target_columns = ["event_type", "task_role", "letter"]
final_column_order = ["onset", "duration", "sample", "event_type", "task_role", "letter", "value"]
log_name = 'sternberg_04_remap_events_log'
template_file = os.path.realpath(os.path.join(bids_root_path, "code/sternberg_remap_event_template_filled.tsv"))

# Set up the logger
log_file_name = f"code/curation_logs/{log_name}.txt"
logger = HedLogger(name=log_name)

# Make the file list and dictionary for _events
bids_files = get_file_list(bids_root_path, extensions=[".tsv"], name_suffix="eventstemp1", exclude_dirs=exclude_dirs)
bids_dict = BidsTsvDictionary("Event files", bids_files, entities=entities)

# Create the template
key_map = KeyMap(key_cols=key_columns, target_cols=target_columns, name='attention_shift_remap')
logger.add("overall",
           f"Creating a key map with key_columns {str(key_columns)} and target_columns {str(target_columns)}")
logger.add("overall", f"Read the template from {os.path.basename(template_file)}")
duplicates = key_map.update(template_file)
if duplicates:
    logger.add("overall", f"This mapping has {len(duplicates)} duplicate key entries", level="ERROR", also_print=True)

for key, file, rowcount, columns in bids_dict.iter_tsv_info():
    filename = bids_dict.get_file_path(key)
    df_bids = get_new_dataframe(filename)
    logger.add(key, f"Remapping {os.path.basename(filename)}")
    df_new, missing = key_map.remap(filename)
    if missing:
        logger.add(key, f"The keys {str(missing)} were not in the map", level="ERROR", also_print=True)
    df_new = df_new.reindex(columns=final_column_order)
    logger.add(key, f"Reordered the columns as {str(final_column_order)}")
    filename_new = filename[:-5] + "2.tsv"
    df_new.to_csv(filename_new, sep='\t', index=False)
    logger.add(key, f"Writing {os.path.basename(filename_new)})")

# Output and save the log
log_string = "\n\nLog output:\n" + logger.get_log_string()
error_string = "\n\nERROR Summary:\n" + logger.get_log_string(level="ERROR")
print(log_string)
print(error_string)
save_path = os.path.join(bids_root_path, log_file_name)
with open(save_path, "w") as fp:
    fp.write(f"{log_file_name} {datetime.datetime.now()}\n")
    fp.write(log_string)
    fp.write(error_string)



Log output:
sternberg_04_remap_events_log: Level None
overall:
	[ Creating a key map with key_columns ['value'] and target_columns ['event_type', 'task_role', 'letter']]
	[ Read the template from sternberg_remap_event_template_filled.tsv]
sub-001_ses-01_run-1:
	[ Remapping sub-001_ses-01_task-WorkingMemory_run-1_eventstemp1.tsv]
	[ Reordered the columns as ['onset', 'duration', 'sample', 'event_type', 'task_role', 'letter', 'value']]
	[ Writing sub-001_ses-01_task-WorkingMemory_run-1_eventstemp2.tsv)]
sub-001_ses-01_run-2:
	[ Remapping sub-001_ses-01_task-WorkingMemory_run-2_eventstemp1.tsv]
	[ Reordered the columns as ['onset', 'duration', 'sample', 'event_type', 'task_role', 'letter', 'value']]
	[ Writing sub-001_ses-01_task-WorkingMemory_run-2_eventstemp2.tsv)]
sub-001_ses-01_run-3:
	[ Remapping sub-001_ses-01_task-WorkingMemory_run-3_eventstemp1.tsv]
	[ Reordered the columns as ['onset', 'duration', 'sample', 'event_type', 'task_role', 'letter', 'value']]
	[ Writing sub-001_ses-0